In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy.io as sio
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import load_iris

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [9]:
'''
    fisher for 2 datasets
    dataset_class: dictionary of dataset (key=label, val=dataset)
    u_class: dictionary of means (key=label, val=u)
'''

def LDA_2_aimd( dataset, label_true, aim_d ):
    
    n = len(dataset)
    print("dataset shape:", dataset.shape)
    
    #get the label for classify
    label = []
    for i in label_true:
        label.append( i[0] )
    label = np.array( label )
    print("label shape:", label.shape)
    label = list( set(label) )
    
    #merge the same_label samples, store them into a dictionary
    dataset_class = {}
    for i in label:
        nw_data = np.array( [ dataset[j] for j in range(n) if label_true[j] == i ] )
        dataset_class[i] = nw_data
    
    u = np.mean( dataset, axis = 0 )
    u_class = {}
    
    for i in label:
        u_class[i] = np.mean( dataset_class[i], axis = 0 )
    
    St = np.zeros( (len(u),len(u)) )
    St = np.dot( ( dataset - u ).T, (dataset - u) )
    print("St shape:", St.shape)
    
    Sw = np.zeros( (len(u),len(u)) )
    for i in label:
        Sw += np.dot( (dataset_class[i]-u_class[i]).T, (dataset_class[i]-u_class[i]) )
    print("Sw shape:", Sw.shape)
    
    Sb = np.zeros( (len(u),len(u)) )
    Sb = St - Sw
    print("Sb shape:",Sb.shape)
    
    eig_vals, eig_vecs = np.linalg.eig( np.dot( np.linalg.pinv(Sw), Sb ) )

    sorted_indices = np.argsort( eig_vals )
    W = eig_vecs[ :, sorted_indices[:-aim_d-1:-1] ]
    
    feaset = []
    for i in range(n):
        feaset.append( np.dot(dataset[i], W) )
    return feaset, Sb, Sw, W, dataset_class, u_class

In [3]:
url = 'data/COIL20.mat'
dataset = sio.loadmat(url)
num1 = 72
num2 = 72
__dataset = dataset['fea'][216:216+num1]
label_true = dataset['gnd'][216+num1:216+num1+num2]

In [10]:
aim_d = 32
feaset, Sb, Sw, W, dataset_calss, u_class = LDA_2_aimd( __dataset, label_true, aim_d )

dataset shape: (72, 1024)
label shape: (72,)
St shape: (1024, 1024)
Sw shape: (1024, 1024)
Sb shape: (1024, 1024)


(8.269912490720877+0j)


In [41]:
iris = load_iris()
X = iris.data
y = iris.target
print(X.shape)
print(y.shape)

(150, 4)
(150,)


In [11]:
lda = LinearDiscriminantAnalysis( n_components = 32 )
X = __dataset
y = np.array( [ i[0] for i in label_true ] )
lda.fit( X, y )
X_new = lda.transform( X )
print(X_new.shape)

(144, 1)


D:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
